# BERT NE recognition FACILITY

## 1)  Preprocess / format data

In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import keras
import nltk
import spacy

Using TensorFlow backend.


In [3]:
from nltk import ne_chunk, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
import gensim.models.word2vec as w2v
from gensim import logging
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as  pd
import seaborn as sns
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import json
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import MWETokenizer

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chantana/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## 2) load keyword: FAC from text

In [56]:
word_files = ['./keywords/facility.txt']
word_list_fac = []
for word_file in word_files:
    with open(word_file) as f:
        lines = f.readlines()
    word_list2 = [x.strip().lower() for x in lines] 
    
    word_list_fac.extend(word_list2)
word_list_fac[0:5]

['24-hour receptionist',
 'laundry service',
 'luggage storage',
 'tours',
 'parking']

In [57]:
#remove duplicates
def ordered_set(in_list):
    out_list = []
    added = set()
    for val in in_list:
        if not val in added:
            out_list.append(val)
            added.add(val)
    return out_list

In [58]:
word_list_fac = ordered_set(word_list_fac)

In [59]:
mw_list = []

word_list = []       
for x in word_list_fac:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
    else:
        word_list.extend(ws)
            
mw_list = ordered_set(mw_list)
mw_list[0:5]

[('24-hour', 'receptionist'),
 ('laundry', 'service'),
 ('luggage', 'storage'),
 ('meeting', 'facilities'),
 ('free', 'wifi')]

In [60]:
word_list[:5]

['tours', 'parking', 'restaurant', 'breakfast', 'garden']

In [61]:
my_idx = {}
for i,w in enumerate(mw_list):
    my_idx[" ".join(w)] = i
print(len(my_idx))

165


In [62]:
word_list = ordered_set(word_list)

In [63]:

for i,w in enumerate(word_list):

    my_idx[w] = i
print(len(my_idx))

247


In [64]:
for i in mw_list:
    if i == ('Free','WiFi'):
        print('found')

for i in word_list:
    if i == 'Refrigerator':
        print('found')

In [71]:
mwe = MWETokenizer(mw_list,separator=' ')

In [66]:
reviews = pd.read_csv("./keywords/facility.csv")

In [72]:
if 'refrigerator' in word_list:
    print("found")

found


In [67]:
reviews.shape

(13522, 1)

In [68]:
reviews.head()

,Facility
0,"For the comfort and convenience of guests,Bang..."
1,"For the comfort and convenience of guests,Baan..."
2,"For the comfort and convenience of guests,Siam..."
3,"For the comfort and convenience of guests,Yota..."
4,"For the comfort and convenience of guests,Chaw..."


In [46]:
reviews = reviews.dropna()
reviews = reviews.reset_index(drop=True)

In [69]:
def spans(txt):
    tokens=mwe.tokenize(word_tokenize(txt))
    offset = 0
    for token in tokens:
        offset = txt.find(token.lower(), offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [80]:
from tqdm import tqdm_notebook as tqdm
all_item = []

for i in tqdm(range(len(reviews))):
    word_ls = []
    #print("sentence:",i+1)
    
    for token in spans(reviews.Facility[i]):
        #print(token)
        #assert token[0]==reviews.location[i][token[1]:token[2]]
        my_tuple = token[0]
        #print("###",token[0].lower())
        
        #my_tuples = ' , '.join(map(str, my_tuple))
        
         
        if token[0].lower() in word_list_fac:
            
            
            #word_ls.append(my_tuple)
            #lis_lo = nltk.pos_tag(word_ls),"FACILITY"
            #print(' , '.join(map(str, lis_lo)))
            
            #my_pos = nltk.pos_tag([my_tuple])[0][1]
            #new_item =  dict({'Sentence #': i+1, 'Tag' : 'FACILITY', 'Word': my_tuple,'POS': my_pos})
            
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-FAC']*len(pos_list)
            tag_list[0] = 'B-FAC' 
            
             
            for s,p,t in zip(subwords,pos_list,tag_list):   
                if type(p) == list:
                    p = p[0][1]
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s.lower(),'POS': p})
                all_item.append(new_item)
                
                 

        else:
            #print(type(my_tuple))
            my_pos = nltk.pos_tag([my_tuple])[0][1]
            new_item = dict({'Sentence #': i+1, 'Tag' : 'O', 'Word': my_tuple.lower(),'POS': my_pos})
            all_item.append(new_item)
            
            #if i+1 == 5177:
                #print(new_item)
             
            #print(nltk.pos_tag([my_tuple]),',','O')
        #print(new_item)
        #if not(my_pos == '.' or  my_pos == ',' or my_pos == ':' or my_pos == '(' or my_pos == ')') :
            
            #all_item.append(new_item) 

        

        
    

          

## 3) save preproc data

In [27]:
file_csv = 'data-fac-bert.csv'
with open(file_csv, 'w') as csv_file:
    csv_file.write('Sentence #*Word*POS*Tag\n')
    for item in all_item:
        
        #print(item['POS'])    
        csv_file.write(str(item['Sentence #'])+'*'+item['Word']+'*'+item['POS']+'*'+item['Tag']+'\n')
csv_file.close()

## 4) Load preprocessed data

In [81]:
import pandas as pd
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
#load data
# explore data
file_csv = 'data-fac-bert.csv'
data = pd.read_csv(file_csv, sep='*',encoding="utf-8").fillna(method="ffill")

data.head(25)
 


,Sentence #,Word,POS,Tag
0,1,for,IN,O
1,1,the,DT,O
2,1,comfort,NN,O
3,1,and,CC,O
4,1,convenience,NN,O
5,1,of,IN,O
6,1,guests,NNS,O
7,1,",",",",O
8,1,bangkok,NN,O
9,1,test,NN,O


### 5) BERT trained-formating

In [82]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [83]:
 
#concat sentence
getter = SentenceGetter(data)

In [377]:
##word_list = [ [s[0] for s in sent] for sent in getter.sentences] 

In [84]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

u'for the comfort and convenience of guests , bangkok test hotel offers Desk , Hairdryer , In-room safe'

In [85]:
print(len(sentences))

13522


In [86]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(len(labels[0]))

18


In [88]:
print(len(labels))

13522


In [89]:
tags_vals = list(set(data["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals) }

In [92]:
def convert_word2idx (text_list):
    
    values = []
    #print(text_list)
     
    for word in text_list:
        if word in my_idx:
            values.append(my_idx[word])
        else:
            if word.lower() in embeddings_index:
                v = len(my_idx)+embeddings_index[word.lower()][0] 
            else:
                val = list(embeddings_index.keys())[0]
                embeddings_index[word.lower()] = (len(embeddings_index),[0.5]*len(val[1]))
                v = len(my_idx) +len(embeddings_index)
            values.append(v)
            
    return values

def convert_idx2word (idx_list):
    words_list = []
    for idx in idx_list:
        found = False
        for key, value in my_idx.items(): 
            if idx == value:
                words_list.append(key)
                found = True
                break
        if not found:
            for key, value in embeddings_index.items(): 
                if idx == value[0]+len(my_idx):
                    words_list.append(key)

                    break

            
         
        
    return words_list


# Training

In [93]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [99]:
MAX_LEN = 128
bs = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0) 

u'TITAN Xp'

In [100]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

2019-03-18 13:47:53,613 : INFO : loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/chantana/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


mwe = MWETokenizer(mw_list,separator=' ')

tokenized_texts = [mwe.tokenize(word_tokenize(sent)) for sent in sentences]


print(tokenized_texts[0],len(tokenized_texts[0]))

In [96]:
tokenized_texts = [ tokenizer.tokenize(sent)   for sent in sentences]
print(tokenized_texts[0])

[u'for', u'the', u'comfort', u'and', u'convenience', u'of', u'guests', u',', u'bangkok', u'test', u'hotel', u'offers', u'desk', u',', u'hair', u'##dry', u'##er', u',', u'in', u'-', u'room', u'safe']


In [97]:
sentences[0]

u'for the comfort and convenience of guests , bangkok test hotel offers Desk , Hairdryer , In-room safe'

In [98]:
print(tokenized_texts[0])

[u'for', u'the', u'comfort', u'and', u'convenience', u'of', u'guests', u',', u'bangkok', u'test', u'hotel', u'offers', u'desk', u',', u'hair', u'##dry', u'##er', u',', u'in', u'-', u'room', u'safe']


In [263]:
def convert_word2idx (text_list):
    
    values = []
    #print(text_list)
    start = len(tokenizer.vocab)
    for word in text_list:
        
        if tokenizer.vocab.has_key(word):
            values.append(tokenizer.vocab[word])
        else:
            #print(word,"not found")
            tokenizer.vocab[word] = start
            values.append(tokenizer.vocab[word])
            start = start + 1
        
            
    return values


In [315]:
len(tokenizer.vocab)

42972

In [208]:
tokenizer_save =  tokenizer


In [241]:
tokenizer.vocab 

OrderedDict([(u'[PAD]', 0),
             (u'[unused0]', 1),
             (u'[unused1]', 2),
             (u'[unused2]', 3),
             (u'[unused3]', 4),
             (u'[unused4]', 5),
             (u'[unused5]', 6),
             (u'[unused6]', 7),
             (u'[unused7]', 8),
             (u'[unused8]', 9),
             (u'[unused9]', 10),
             (u'[unused10]', 11),
             (u'[unused11]', 12),
             (u'[unused12]', 13),
             (u'[unused13]', 14),
             (u'[unused14]', 15),
             (u'[unused15]', 16),
             (u'[unused16]', 17),
             (u'[unused17]', 18),
             (u'[unused18]', 19),
             (u'[unused19]', 20),
             (u'[unused20]', 21),
             (u'[unused21]', 22),
             (u'[unused22]', 23),
             (u'[unused23]', 24),
             (u'[unused24]', 25),
             (u'[unused25]', 26),
             (u'[unused26]', 27),
             (u'[unused27]', 28),
             (u'[unused28]', 29),
     

In [101]:
print(mw_list[:5])
for i in mw_list:
    if " ".join(i) == '@ Home Executive Apartment':
        print('found')

[('24-hour', 'receptionist'), ('laundry', 'service'), ('luggage', 'storage'), ('meeting', 'facilities'), ('free', 'wifi')]


In [102]:
def append_vocab(vocab_tuple_list):
    start = len(tokenizer.vocab)
    
    for v in vocab_tuple_list:
        ww = " ".join(v)
        if ww not in tokenizer.vocab:
            #print(v)
            tokenizer.vocab[ww] = start
            print(ww)
            start += 1
    return tokenizer.vocab


In [103]:
tokens_ids = [  tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts]


In [106]:
 print(len(tokens_ids[0]),len(labels[0]))
 

22 18


In [107]:
#input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
input_ids = pad_sequences(tokens_ids,
                          maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")
#input_ids = pad_sequences([convert_word2idx(txt) for txt in tokenized_texts],
                          #maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")

In [108]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")


In [109]:
print(MAX_LEN)
for i in tokens_ids:
    if len(i) > MAX_LEN:
        #print(tokens_ids)
        print("more")

128


In [110]:
t_list = [[tag2idx.get(l) for l in lab] for lab in labels]
len(t_list[0])

18

In [111]:
print(input_ids[0])

[ 2005  1996  7216  1998 15106  1997  6368  1010 12627  3231  3309  4107
  4624  1010  2606 21190  2121  1010  1999  1011  2282  3647     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [112]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="int64", truncating="post")

In [113]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [114]:
#split train test
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.3)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.3)

In [115]:
#change to torhc tensor
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [116]:
print(tr_inputs.shape)
print(tr_masks.shape)
print(tr_tags.shape)

torch.Size([9465, 128])
torch.Size([9465, 128])
torch.Size([9465, 128])


In [117]:
print(val_inputs.shape)
print(val_masks.shape)
print(val_tags.shape)

torch.Size([4057, 128])
torch.Size([4057, 128])
torch.Size([4057, 128])


In [118]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [119]:
print(len(tag2idx))

3


In [120]:
model = BertForTokenClassification.from_pretrained(u"bert-base-uncased", num_labels=len(tag2idx))

2019-03-18 13:49:15,150 : INFO : loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/chantana/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
2019-03-18 13:49:15,152 : INFO : extracting archive file /home/chantana/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmplIt6Sc
2019-03-18 13:49:18,988 : INFO : Model config {
  "attention_probs_dropout_prob": 0.1, 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "type_vocab_size": 2, 
  "vocab_size": 30522
}

2019-03-18 13:49:22,296 : INFO : Weights of BertForTokenClass

In [121]:
model.cuda();

In [122]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [123]:
#Fine tune BERT
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat) 

In [124]:
epochs = 5
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.208591696693
Validation loss: 0.142126746769
Validation Accuracy: 0.922194326539


Epoch:  20%|██        | 1/5 [05:13<20:52, 313.16s/it]

F1-Score: 0.202129429379
Train loss: 0.109954242003
Validation loss: 0.0608095850385
Validation Accuracy: 0.936126789083


Epoch:  40%|████      | 2/5 [09:48<15:05, 301.90s/it]

...

F1-Score: 0.402615809235
Train loss: 0.0458990340546
Validation loss: 0.0355344091378
Validation Accuracy: 0.938551321222


Epoch:  80%|████████  | 4/5 [18:55<04:47, 287.40s/it]

F1-Score: 0.419168897571
Train loss: 0.0373008890023
Validation loss: 0.0295377021161
Validation Accuracy: 0.939692352909


Epoch: 100%|██████████| 5/5 [24:07<00:00, 294.88s/it]

F1-Score: 0.434325574636


In [125]:
#evaluate model
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
print(len(valid_dataloader))
for batch in tqdm(valid_dataloader):
    #print(len(batch))
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

  0%|          | 1/254 [00:00<00:40,  6.27it/s]

254


100%|██████████| 254/254 [00:36<00:00,  6.33it/s]


Validation loss: 0.0295377021161
Validation Accuracy: 0.939692352909
Validation F1-Score: 0.434325574636


In [293]:
pred_tags

[[u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O'],
 [u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
 

for lab in b_labels:
    for l in lab:
        print(idx2tag[int(l)])
        

In [384]:
[[ idx2tag[int(l)] for l in lab] for lab in b_labels]

[[u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O'],
 [u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',